# Extraction of Frequency Bands and Asymetry Data


In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import mne
import matplotlib.pyplot as plt
%matplotlib inline

### Create marker list
- Here the idea is to seperae the EEG data recorded for one participant into t(s) epochs (example t = 15s). 
- This yeilds more sampltes to train, validate and test our models

In [2]:
#Create marker list
def get_markers(epoch_length=15):
    sampling_rate = 125 # 125 Hz
    min_frac = (epoch_length/60.0)
    step_minutes = np.arange(0,6,min_frac) # Start - 0 min, total_recording_time - 6 min, step = 15s(i.e. 0.25 min)
    print(f"{step_minutes=}")
    step_minutes = np.expand_dims(step_minutes * sampling_rate * 60, axis=1)
    markers = np.concatenate( [step_minutes, np.zeros( step_minutes.shape ), np.ones( step_minutes.shape ) ], axis=1  ).astype(np.int64)
    return markers

epoch_length = 15 # This is 15 seconds, can be reduced to 10s if we need more data
markers = get_markers(epoch_length=epoch_length)
len(markers)

step_minutes=array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 ,
       2.75, 3.  , 3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75, 5.  , 5.25,
       5.5 , 5.75])


24

### Convert to mne.io.RawArray

In [3]:
#Convert a Pandas df to mne.io.RawArray
def dataframe_to_raw(dataframe, sfreq):
    ch_names = list(dataframe.columns)
    ch_types = ['eeg'] * (len(dataframe.columns))
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

    dataframe = dataframe.T  # mne looks at the tranpose() format
    dataframe *= 1e-6  # convert from uVolts to Volts (mne assumes Volts data)

    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq, verbose=False)

    raw = mne.io.RawArray(dataframe, info,verbose=False)
    raw.set_montage(ten_twenty_montage)
    
    return raw

### Read labels CSV

In [4]:
# Read the csv file with lables and put id, lable into a dict.
def read_labels(file_path):
    label_df = pd.read_csv(file_path)
    data = label_df[['id', 'label']]
    data = data.to_numpy()
    dict_label = dict()

    for i in range(len(data)):
        dict_label[data[i][0]] = data[i][1]
    return dict_label

label_file_path = '../data/labels/PSS10-Sheet1.csv'
dict_label = read_labels(label_file_path)    
print(dict_label)

{1: 1, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 1, 17: 1}


### Export PSD featurs from all ch_16_csv files

In [5]:
# Reference1 : https://github.com/akraradets/EEG_ChronicStress/blob/main/2-ChronicStress/3-export-feature.ipynb
# Reference2 : https://github.com/chaklam-silpasuwanchai/Python-for-EEG/blob/main/03%20-%20Asymmetry%20%2B%20DEAP.ipynb

def export_features_to_csv(in_dir_path, out_dir_path, file_name):

    #dir_path = '../data/ch_16_csv'
    #f_name = 'conv_exp01.csv'
    f_path = in_dir_path + '/' + file_name
    # Read csv
    in_df = pd.read_csv(f_path)

    f_idx = file_name[3:]
    f_idx = f_idx[:-4]
    f_id = int(f_idx)
    label = dict_label[f_id]
    print(f_id, label)

    # df to mne.io.RawArray
    raw = dataframe_to_raw(in_df, 125)

    #Epochs
    epochs = mne.Epochs(raw, markers, tmin=0, tmax=8, baseline=None, verbose=False) #Our recording time is 6min thus as tmax 8min is used
    spectrum = epochs.compute_psd(method="welch", n_fft=125)  # Get Spectrum object from epochs

    #The shape of PSDs is (n_epochs,n_channels,n_freqs) and FREQs is (n_freqs)
    psds, freqs = spectrum.get_data(return_freqs=True) 
    # psds.shape, freqs.shape, (24, 16, 63) (63,)
    psds = 10 * np.log10(psds)

    band_names = np.array(['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Slow', 'Low_beta', 'Relative_gamma'])
    mean_band_names = np.array(['mean_Delta', 'mean_Theta', 'mean_Alpha', 'mean_Beta', 
                                'mean_Gamma', 'mean_Slow', 'mean_Low_beta', 'mean_Relative_gamma'])
    #filter_list = [[1,3],[4,7],[8,12],[13,30],[30,43], [4,13], [13,17]]

    #Take the mean of all channels
    mean_psds = psds.mean(axis=1)
    #mean_psds.shape, (24, 63)

    #Band mean extraction
    delta = mean_psds[:, 1:3]
    theta = mean_psds[:, 4:7]
    alpha = mean_psds[:, 8:12]
    beta  = mean_psds[:, 13:30]
    gamma = mean_psds[:, 30:43]
    slow = mean_psds[:, 4:13]
    low_beta = mean_psds[:, 13:17]

    #For each band take the mean of the power
    delta = delta.mean(axis=1).reshape(-1, 1)
    theta = theta.mean(axis=1).reshape(-1, 1)
    alpha = alpha.mean(axis=1).reshape(-1, 1)
    beta  = beta.mean(axis=1).reshape(-1, 1)
    gamma = gamma.mean(axis=1).reshape(-1, 1)
    slow = slow.mean(axis=1).reshape(-1, 1)
    low_beta = low_beta.mean(axis=1).reshape(-1, 1)
    relative_gamma = slow/gamma

    #Stack them together
    _temp = [delta, theta, alpha, beta, gamma, slow, low_beta, relative_gamma]
    all_bands = np.hstack(_temp)
    #all_bands.shape , (24, 7)

    out_df = pd.DataFrame(all_bands, columns = mean_band_names.tolist())

    # Jo's channels ['Fp1','Fp2','F3','F4','F7','F8','C3','C4','T3','T4','T5','T6','P3','P4','O1','O2']
    # Our channels['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']

    # Band extraction
    channel_list = ['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']

    for i in range(len(channel_list)):
        new_band_names = []
        for band_name in band_names:
            new_name = channel_list[i] + '_' + band_name
            new_band_names.append(new_name)

        delta = psds[:, i, 1:3]
        theta = psds[:, i, 4:7]
        alpha = psds[:, i, 8:12]
        beta  = psds[:, i, 13:30]
        gamma = psds[:, i, 30:43]
        slow = psds[:, i, 4:13]
        low_beta = psds[:, i, 13:17]

        #For each band take the mean of the power
        delta = delta.mean(axis=1).reshape(-1, 1)
        theta = theta.mean(axis=1).reshape(-1, 1)
        alpha = alpha.mean(axis=1).reshape(-1, 1)
        beta  = beta.mean(axis=1).reshape(-1, 1)
        gamma = gamma.mean(axis=1).reshape(-1, 1)
        slow = slow.mean(axis=1).reshape(-1, 1)
        low_beta = low_beta.mean(axis=1).reshape(-1, 1)
        relative_gamma = slow/gamma

        #Stack them together
        _temp = [delta, theta, alpha, beta, gamma, slow, low_beta, relative_gamma]
        all_bands = np.hstack(_temp)
        temp_df = pd.DataFrame(all_bands, columns = new_band_names)
        out_df = pd.concat([out_df, temp_df], axis=1, join='inner')
    
    # alpha_frontal asymetry
    a_f4 = psds[:, 4, 8:12]
    a_f4 = a_f4.mean(axis=1).reshape(-1, 1)
    a_f3 = psds[:, 3, 8:12]
    a_f3 = a_f3.mean(axis=1).reshape(-1, 1)

    alpha_frontal = ((a_f4 - a_f3) / (a_f4 + a_f3)).reshape(1,-1)
    out_df['alpha_frontal'] = alpha_frontal[0]

    # alpha_temporal asymetry
    a_t4 = psds[:, 9, 8:12]
    a_t4 = a_t4.mean(axis=1).reshape(-1, 1)
    a_t3 = psds[:, 6, 8:12]
    a_t3 = a_t3.mean(axis=1).reshape(-1, 1)

    alpha_temporal = ((a_t4 - a_t3) / (a_t4 + a_t3)).reshape(1,-1)
    out_df['alpha_temporal'] = alpha_temporal[0]

    # alpha_asymmetry
    alpha_asymmetry = alpha_frontal[0] + alpha_temporal[0]
    out_df['alpha_asymmetry'] = alpha_asymmetry

    # beta_frontal asymetry
    b_f4 = psds[:, 4, 13:30] # F4 index is 4
    b_f4 = b_f4.mean(axis=1).reshape(-1, 1)
    b_f3 = psds[:, 3, 13:30] # F3 index is 3
    b_f3 = b_f3.mean(axis=1).reshape(-1, 1)

    beta_frontal = ((b_f4 - b_f3) / (b_f4 + b_f3)).reshape(1,-1)
    out_df['beta_frontal'] = beta_frontal[0]

    # beta_temporal asymetry
    b_t4 = psds[:, 9, 13:30] # T4 index is 9
    b_t4 = b_t4.mean(axis=1).reshape(-1, 1)
    b_t3 = psds[:, 6, 13:30] # T3 index is 6
    b_t3 = b_t3.mean(axis=1).reshape(-1, 1)

    beta_temporal = ((b_t4 - b_t3) / (b_t4 + b_t3)).reshape(1,-1)
    out_df['beta_temporal'] = beta_temporal[0]

    # beta_asymmetry
    beat_asymmetry = beta_frontal[0] + beta_temporal[0]
    out_df['beat_asymmetry'] = beat_asymmetry

    out_df['participant_id'] = f_id
    out_df['label'] = label

    out_file_path = out_dir_path + '/' + file_name
    out_df.to_csv(out_file_path,index=False)


In [6]:
in_dir_path = '../data/ch_16_csv'
out_dir_path = '../data/psd_features_csv'
file_names = ['exp01.csv', 'exp02.csv', 'exp03.csv', 'exp04.csv', 'exp05.csv', 'exp06.csv', 
              'exp07.csv', 'exp08.csv', 'exp09.csv', 'exp10.csv', 'exp11.csv', 'exp12.csv', 
              'exp13.csv', 'exp14.csv', 'exp15.csv', 'exp16.csv', 'exp17.csv']

for file_name in file_names:
    export_features_to_csv(in_dir_path, out_dir_path, file_name)

1 1
Using data from preloaded Raw for 24 events and 1001 original time points ...
0 bad epochs dropped
Effective window size : 1.000 (s)
2 1
Using data from preloaded Raw for 24 events and 1001 original time points ...
0 bad epochs dropped
Effective window size : 1.000 (s)
3 1
Using data from preloaded Raw for 24 events and 1001 original time points ...
0 bad epochs dropped
Effective window size : 1.000 (s)
4 0
Using data from preloaded Raw for 24 events and 1001 original time points ...
0 bad epochs dropped
Effective window size : 1.000 (s)
5 0
Using data from preloaded Raw for 24 events and 1001 original time points ...
0 bad epochs dropped
Effective window size : 1.000 (s)
6 0
Using data from preloaded Raw for 24 events and 1001 original time points ...
0 bad epochs dropped
Effective window size : 1.000 (s)
7 0
Using data from preloaded Raw for 24 events and 1001 original time points ...
0 bad epochs dropped
Effective window size : 1.000 (s)
8 1
Using data from preloaded Raw for 24 

In [7]:
# Test one PSD feature CSV file
out_file_path = '../data/psd_features_csv/exp10.csv'
df = pd.read_csv(out_file_path)
df.head()

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-99.768528,-115.601980,-116.027096,-121.981084,-125.625050,-115.502713,-118.524726,0.919424,-99.913486,-114.883408,...,-116.224456,0.926290,-0.001216,-0.012249,-0.013465,0.000913,0.001942,0.002855,10,0
1,-101.079159,-116.061275,-116.982246,-120.643720,-125.209438,-116.374295,-117.079246,0.929437,-100.288870,-115.203461,...,-116.300075,0.936851,-0.000209,-0.015908,-0.016118,-0.002772,-0.002171,-0.004943,10,0
2,-104.177636,-115.654665,-115.540488,-121.297468,-125.250074,-115.580408,-118.300836,0.922797,-104.584108,-115.379281,...,-115.274266,0.933023,0.000110,-0.006649,-0.006539,-0.001232,-0.003261,-0.004493,10,0
3,-105.295967,-115.685869,-116.597336,-121.681380,-125.243638,-115.766490,-119.054848,0.924330,-105.025209,-115.988855,...,-115.701785,0.938191,0.001047,-0.001451,-0.000404,-0.002001,0.002885,0.000884,10,0
4,-101.000643,-115.040556,-116.136052,-122.150630,-125.484638,-115.854402,-119.346297,0.923256,-99.697458,-113.153722,...,-116.736613,0.938273,-0.004929,-0.022983,-0.027912,-0.002532,-0.000519,-0.003050,10,0


### Export all PSD feature CSV files to one CSV file

In [8]:
dir_path = '../data/psd_features_csv'
file_names_from_second = ['exp02.csv', 'exp03.csv', 'exp04.csv', 'exp05.csv', 'exp06.csv', 
              'exp07.csv', 'exp08.csv', 'exp09.csv', 'exp10.csv', 'exp11.csv', 'exp12.csv', 
              'exp13.csv', 'exp14.csv', 'exp15.csv', 'exp16.csv', 'exp17.csv']
file_path = '../data/psd_features_csv/exp01.csv'
all_df = pd.read_csv(file_path)
all_df.head()

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-110.329965,-121.299482,-123.725032,-131.335399,-134.971232,-123.031043,-128.526178,0.911535,-106.209371,-117.231013,...,-127.567021,0.902035,-0.002908,-0.000649,-0.003557,-0.002668,0.010859,0.008191,1,1
1,-110.383640,-121.158052,-123.636121,-130.643007,-134.449370,-123.097757,-128.670923,0.915570,-107.994728,-120.509328,...,-130.006559,0.915688,-0.005629,-0.008944,-0.014573,-0.001315,0.003450,0.002135,1,1
2,-111.593383,-120.712248,-124.276396,-130.440765,-134.357490,-123.286220,-130.602665,0.917598,-110.782820,-117.801016,...,-131.785838,0.918551,-0.003908,-0.004460,-0.008369,0.000354,0.002001,0.002355,1,1
3,-111.292704,-122.189083,-123.512155,-128.614875,-135.116331,-123.276588,-127.691734,0.912374,-108.605531,-119.022817,...,-129.835990,0.924993,-0.005010,0.010191,0.005181,-0.005576,0.003535,-0.002042,1,1
4,-113.361314,-119.989890,-123.255042,-130.600926,-135.144714,-122.298936,-129.782153,0.904948,-109.577307,-117.077183,...,-131.338841,0.901524,0.001069,-0.002919,-0.001850,-0.007245,0.008795,0.001550,1,1


In [9]:
for file_name in file_names_from_second:
    file_path = dir_path + '/' + file_name
    temp_df = pd.read_csv(file_path)
    frames = [all_df, temp_df]
    all_df = pd.concat(frames,ignore_index=True)

In [10]:
all_df.head(-5)

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-110.329965,-121.299482,-123.725032,-131.335399,-134.971232,-123.031043,-128.526178,0.911535,-106.209371,-117.231013,...,-127.567021,0.902035,-0.002908,-0.000649,-0.003557,-0.002668,0.010859,0.008191,1,1
1,-110.383640,-121.158052,-123.636121,-130.643007,-134.449370,-123.097757,-128.670923,0.915570,-107.994728,-120.509328,...,-130.006559,0.915688,-0.005629,-0.008944,-0.014573,-0.001315,0.003450,0.002135,1,1
2,-111.593383,-120.712248,-124.276396,-130.440765,-134.357490,-123.286220,-130.602665,0.917598,-110.782820,-117.801016,...,-131.785838,0.918551,-0.003908,-0.004460,-0.008369,0.000354,0.002001,0.002355,1,1
3,-111.292704,-122.189083,-123.512155,-128.614875,-135.116331,-123.276588,-127.691734,0.912374,-108.605531,-119.022817,...,-129.835990,0.924993,-0.005010,0.010191,0.005181,-0.005576,0.003535,-0.002042,1,1
4,-113.361314,-119.989890,-123.255042,-130.600926,-135.144714,-122.298936,-129.782153,0.904948,-109.577307,-117.077183,...,-131.338841,0.901524,0.001069,-0.002919,-0.001850,-0.007245,0.008795,0.001550,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,-106.183861,-111.842076,-112.661217,-114.930348,-116.640437,-112.341899,-114.296380,0.963147,-106.965264,-112.120068,...,-117.029047,0.885861,0.002874,0.001699,0.004573,-0.002137,0.000399,-0.001738,17,1
399,-107.267561,-111.890905,-113.581270,-114.550148,-116.529412,-112.930248,-115.023712,0.969114,-109.739237,-111.134677,...,-120.212574,0.924304,-0.001313,0.004682,0.003369,-0.002560,0.000461,-0.002099,17,1
400,-105.462343,-113.128064,-113.194629,-113.695168,-116.755079,-113.001532,-113.404254,0.967851,-107.875827,-114.115695,...,-116.921379,0.909639,0.000248,0.000456,0.000704,-0.002474,0.001370,-0.001104,17,1
401,-104.400169,-111.629116,-112.040812,-114.548587,-116.866835,-111.960661,-113.850860,0.958019,-98.778013,-111.643683,...,-117.992750,0.886588,-0.002806,-0.003916,-0.006721,-0.001995,-0.000334,-0.002329,17,1


In [11]:
all_df.shape

(408, 144)

In [12]:
file_path = '../data/psd_features_csv/all.csv'
all_df.to_csv(file_path,index=False)

In [13]:
# Test all.csv PSD feature CSV file
file_path = '../data/psd_features_csv/all.csv'
df = pd.read_csv(file_path)
df.head()

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-110.329965,-121.299482,-123.725032,-131.335399,-134.971232,-123.031043,-128.526178,0.911535,-106.209371,-117.231013,...,-127.567021,0.902035,-0.002908,-0.000649,-0.003557,-0.002668,0.010859,0.008191,1,1
1,-110.383640,-121.158052,-123.636121,-130.643007,-134.449370,-123.097757,-128.670923,0.915570,-107.994728,-120.509328,...,-130.006559,0.915688,-0.005629,-0.008944,-0.014573,-0.001315,0.003450,0.002135,1,1
2,-111.593383,-120.712248,-124.276396,-130.440765,-134.357490,-123.286220,-130.602665,0.917598,-110.782820,-117.801016,...,-131.785838,0.918551,-0.003908,-0.004460,-0.008369,0.000354,0.002001,0.002355,1,1
3,-111.292704,-122.189083,-123.512155,-128.614875,-135.116331,-123.276588,-127.691734,0.912374,-108.605531,-119.022817,...,-129.835990,0.924993,-0.005010,0.010191,0.005181,-0.005576,0.003535,-0.002042,1,1
4,-113.361314,-119.989890,-123.255042,-130.600926,-135.144714,-122.298936,-129.782153,0.904948,-109.577307,-117.077183,...,-131.338841,0.901524,0.001069,-0.002919,-0.001850,-0.007245,0.008795,0.001550,1,1


In [14]:
temp_list = list()
for i in df.head(0):
    temp_list.append(i)

In [15]:
temp_list

['mean_Delta',
 'mean_Theta',
 'mean_Alpha',
 'mean_Beta',
 'mean_Gamma',
 'mean_Slow',
 'mean_Low_beta',
 'mean_Relative_gamma',
 'Fp1_Delta',
 'Fp1_Theta',
 'Fp1_Alpha',
 'Fp1_Beta',
 'Fp1_Gamma',
 'Fp1_Slow',
 'Fp1_Low_beta',
 'Fp1_Relative_gamma',
 'Fp2_Delta',
 'Fp2_Theta',
 'Fp2_Alpha',
 'Fp2_Beta',
 'Fp2_Gamma',
 'Fp2_Slow',
 'Fp2_Low_beta',
 'Fp2_Relative_gamma',
 'F7_Delta',
 'F7_Theta',
 'F7_Alpha',
 'F7_Beta',
 'F7_Gamma',
 'F7_Slow',
 'F7_Low_beta',
 'F7_Relative_gamma',
 'F3_Delta',
 'F3_Theta',
 'F3_Alpha',
 'F3_Beta',
 'F3_Gamma',
 'F3_Slow',
 'F3_Low_beta',
 'F3_Relative_gamma',
 'F4_Delta',
 'F4_Theta',
 'F4_Alpha',
 'F4_Beta',
 'F4_Gamma',
 'F4_Slow',
 'F4_Low_beta',
 'F4_Relative_gamma',
 'F8_Delta',
 'F8_Theta',
 'F8_Alpha',
 'F8_Beta',
 'F8_Gamma',
 'F8_Slow',
 'F8_Low_beta',
 'F8_Relative_gamma',
 'T3_Delta',
 'T3_Theta',
 'T3_Alpha',
 'T3_Beta',
 'T3_Gamma',
 'T3_Slow',
 'T3_Low_beta',
 'T3_Relative_gamma',
 'C3_Delta',
 'C3_Theta',
 'C3_Alpha',
 'C3_Beta',
 'C3_

In [16]:
#cols = df.columns
#cols
#cols[60:150]

In [17]:
#temp_list = ['mean_Delta', 'mean_Theta', 'mean_Alpha', 'mean_Beta', 'mean_Gamma', 'mean_Slow', 'mean_Low_beta', 'Fp1_Delta', 'Fp1_Theta', 'Fp1_Alpha', 'Fp1_Beta', 'Fp1_Gamma', 'Fp1_Slow', 'Fp1_Low_beta', 'Fp2_Delta', 'Fp2_Theta', 'Fp2_Alpha', 'Fp2_Beta', 'Fp2_Gamma', 'Fp2_Slow', 'Fp2_Low_beta', 'F7_Delta', 'F7_Theta', 'F7_Alpha', 'F7_Beta', 'F7_Gamma', 'F7_Slow', 'F7_Low_beta', 'F3_Delta', 'F3_Theta', 'F3_Alpha', 'F3_Beta', 'F3_Gamma', 'F3_Slow', 'F3_Low_beta', 'F4_Delta', 'F4_Theta', 'F4_Alpha', 'F4_Beta', 'F4_Gamma', 'F4_Slow', 'F4_Low_beta', 'F8_Delta', 'F8_Theta', 'F8_Alpha', 'F8_Beta', 'F8_Gamma', 'F8_Slow', 'F8_Low_beta', 'T3_Delta', 'T3_Theta', 'T3_Alpha', 'T3_Beta', 'T3_Gamma', 'T3_Slow', 'T3_Low_beta', 'C3_Delta', 'C3_Theta', 'C3_Alpha', 'C3_Beta', 'C3_Gamma', 'C3_Slow', 'C3_Low_beta', 'C4_Delta', 'C4_Theta', 'C4_Alpha', 'C4_Beta', 'C4_Gamma', 'C4_Slow', 'C4_Low_beta', 'T4_Delta', 'T4_Theta', 'T4_Alpha', 'T4_Beta', 'T4_Gamma', 'T4_Slow', 'T4_Low_beta', 'T5_Delta', 'T5_Theta', 'T5_Alpha', 'T5_Beta', 'T5_Gamma', 'T5_Slow', 'T5_Low_beta', 'P3_Delta', 'P3_Theta', 'P3_Alpha', 'P3_Beta', 'P3_Gamma', 'P3_Slow', 'P3_Low_beta', 'P4_Delta', 'P4_Theta', 'P4_Alpha', 'P4_Beta', 'P4_Gamma', 'P4_Slow', 'P4_Low_beta', 'T6_Delta', 'T6_Theta', 'T6_Alpha', 'T6_Beta', 'T6_Gamma', 'T6_Slow', 'T6_Low_beta', 'O1_Delta', 'O1_Theta', 'O1_Alpha', 'O1_Beta', 'O1_Gamma', 'O1_Slow', 'O1_Low_beta', 'O2_Delta', 'O2_Theta', 'O2_Alpha', 'O2_Beta', 'O2_Gamma', 'O2_Slow', 'O2_Low_beta', 'alpha_frontal', 'alpha_temporal', 'alpha_asymmetry', 'beta_frontal', 'beta_temporal', 'beat_asymmetry', 'participant_id', 'label']

In [18]:
len(temp_list)

144

In [19]:
# a = np.array([[2],[4],[6]])
# a.shape

In [20]:
# b = np.array([[1],[2],[3]])

# a/b